# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 27 2023 2:35PM,258281,19,ADV80012933,"AdvaGen Pharma, Ltd",Released
1,Feb 27 2023 2:35PM,258281,19,ADV80012934,"AdvaGen Pharma, Ltd",Released
2,Feb 27 2023 2:35PM,258281,19,ADV80012919,"AdvaGen Pharma, Ltd",Released
3,Feb 27 2023 2:35PM,258281,19,ADV80012920,"AdvaGen Pharma, Ltd",Released
4,Feb 27 2023 2:35PM,258281,19,ADV80012929,"AdvaGen Pharma, Ltd",Released
5,Feb 27 2023 2:35PM,258281,19,ADV80012930,"AdvaGen Pharma, Ltd",Released
6,Feb 27 2023 2:35PM,258281,19,ADV80012931,"AdvaGen Pharma, Ltd",Released
7,Feb 27 2023 2:35PM,258281,19,ADV80012932,"AdvaGen Pharma, Ltd",Released
8,Feb 27 2023 2:35PM,258281,19,ADV80012913,"AdvaGen Pharma, Ltd",Released
9,Feb 27 2023 2:35PM,258281,19,ADV80012914,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
50,258277,Released,43
51,258278,Released,8
52,258279,Released,1
53,258280,Released,16
54,258281,Released,21


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
258277,NaN,NaN,NaN,43.0
258278,NaN,NaN,NaN,8.0
258279,NaN,NaN,NaN,1.0
258280,NaN,NaN,NaN,16.0
258281,NaN,NaN,NaN,21.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
258149,1.0,0.0,15.0,2.0
258153,0.0,0.0,31.0,0.0
258162,0.0,0.0,0.0,58.0
258166,0.0,10.0,7.0,0.0
258188,0.0,0.0,24.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
258149,1.0,0,15,2
258153,0.0,0,31,0
258162,0.0,0,0,58
258166,0.0,10,7,0
258188,0.0,0,24,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,258149,1.0,0,15,2
1,258153,0.0,0,31,0
2,258162,0.0,0,0,58
3,258166,0.0,10,7,0
4,258188,0.0,0,24,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,258149,1.0,,15,2
1,258153,0.0,,31,
2,258162,0.0,,,58
3,258166,0.0,10,7,
4,258188,0.0,,24,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 27 2023 2:35PM,258281,19,"AdvaGen Pharma, Ltd"
21,Feb 27 2023 2:31PM,258280,10,ISDIN Corporation
37,Feb 27 2023 2:23PM,258277,10,ISDIN Corporation
80,Feb 27 2023 2:23PM,258278,10,Bio-PRF
88,Feb 27 2023 2:22PM,258279,18,"Emersa Waterbox, LLC"
89,Feb 27 2023 2:08PM,258275,10,Yusen – 3D Matrix
90,Feb 27 2023 2:01PM,258274,10,"Nextsource Biotechnology, LLC"
93,Feb 27 2023 1:56PM,258273,10,Emerginnova
95,Feb 27 2023 1:48PM,258272,16,Sartorius Bioprocess Solutions
96,Feb 27 2023 1:30PM,258266,19,"GCH Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,Feb 27 2023 2:35PM,258281,19,"AdvaGen Pharma, Ltd",0.0,,,21
1,Feb 27 2023 2:31PM,258280,10,ISDIN Corporation,0.0,,,16
2,Feb 27 2023 2:23PM,258277,10,ISDIN Corporation,0.0,,,43
3,Feb 27 2023 2:23PM,258278,10,Bio-PRF,0.0,,,8
4,Feb 27 2023 2:22PM,258279,18,"Emersa Waterbox, LLC",0.0,,,1
5,Feb 27 2023 2:08PM,258275,10,Yusen – 3D Matrix,0.0,,,1
6,Feb 27 2023 2:01PM,258274,10,"Nextsource Biotechnology, LLC",0.0,,,3
7,Feb 27 2023 1:56PM,258273,10,Emerginnova,0.0,,,2
8,Feb 27 2023 1:48PM,258272,16,Sartorius Bioprocess Solutions,0.0,,,1
9,Feb 27 2023 1:30PM,258266,19,"GCH Granules USA, Inc.",0.0,,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 27 2023 2:35PM,258281,19,"AdvaGen Pharma, Ltd",21,,
1,Feb 27 2023 2:31PM,258280,10,ISDIN Corporation,16,,
2,Feb 27 2023 2:23PM,258277,10,ISDIN Corporation,43,,
3,Feb 27 2023 2:23PM,258278,10,Bio-PRF,8,,
4,Feb 27 2023 2:22PM,258279,18,"Emersa Waterbox, LLC",1,,
5,Feb 27 2023 2:08PM,258275,10,Yusen – 3D Matrix,1,,
6,Feb 27 2023 2:01PM,258274,10,"Nextsource Biotechnology, LLC",3,,
7,Feb 27 2023 1:56PM,258273,10,Emerginnova,2,,
8,Feb 27 2023 1:48PM,258272,16,Sartorius Bioprocess Solutions,1,,
9,Feb 27 2023 1:30PM,258266,19,"GCH Granules USA, Inc.",2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 27 2023 2:35PM,258281,19,"AdvaGen Pharma, Ltd",21,,
1,Feb 27 2023 2:31PM,258280,10,ISDIN Corporation,16,,
2,Feb 27 2023 2:23PM,258277,10,ISDIN Corporation,43,,
3,Feb 27 2023 2:23PM,258278,10,Bio-PRF,8,,
4,Feb 27 2023 2:22PM,258279,18,"Emersa Waterbox, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 27 2023 2:35PM,258281,19,"AdvaGen Pharma, Ltd",21.0,NaN,NaN
1,Feb 27 2023 2:31PM,258280,10,ISDIN Corporation,16.0,NaN,NaN
2,Feb 27 2023 2:23PM,258277,10,ISDIN Corporation,43.0,NaN,NaN
3,Feb 27 2023 2:23PM,258278,10,Bio-PRF,8.0,NaN,NaN
4,Feb 27 2023 2:22PM,258279,18,"Emersa Waterbox, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 27 2023 2:35PM,258281,19,"AdvaGen Pharma, Ltd",21.0,0.0,0.0
1,Feb 27 2023 2:31PM,258280,10,ISDIN Corporation,16.0,0.0,0.0
2,Feb 27 2023 2:23PM,258277,10,ISDIN Corporation,43.0,0.0,0.0
3,Feb 27 2023 2:23PM,258278,10,Bio-PRF,8.0,0.0,0.0
4,Feb 27 2023 2:22PM,258279,18,"Emersa Waterbox, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5680941,358.0,84.0,0.0
12,258245,0.0,1.0,0.0
15,774611,37.0,29.0,78.0
16,516473,2.0,0.0,0.0
18,258279,1.0,0.0,0.0
19,3357131,45.0,25.0,10.0
21,258234,1.0,0.0,0.0
22,516510,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5680941,358.0,84.0,0.0
1,12,258245,0.0,1.0,0.0
2,15,774611,37.0,29.0,78.0
3,16,516473,2.0,0.0,0.0
4,18,258279,1.0,0.0,0.0
5,19,3357131,45.0,25.0,10.0
6,21,258234,1.0,0.0,0.0
7,22,516510,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,358.0,84.0,0.0
1,12,0.0,1.0,0.0
2,15,37.0,29.0,78.0
3,16,2.0,0.0,0.0
4,18,1.0,0.0,0.0
5,19,45.0,25.0,10.0
6,21,1.0,0.0,0.0
7,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,358.0
1,12,Released,0.0
2,15,Released,37.0
3,16,Released,2.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,21,22
Status,,,,,,,,
Completed,0.0,0.0,78.0,0.0,0.0,10.0,0.0,0.0
Executing,84.0,1.0,29.0,0.0,0.0,25.0,0.0,0.0
Released,358.0,0.0,37.0,2.0,1.0,45.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,21,22
0,Completed,0.0,0.0,78.0,0.0,0.0,10.0,0.0,0.0
1,Executing,84.0,1.0,29.0,0.0,0.0,25.0,0.0,0.0
2,Released,358.0,0.0,37.0,2.0,1.0,45.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,21,22
0,Completed,0.0,0.0,78.0,0.0,0.0,10.0,0.0,0.0
1,Executing,84.0,1.0,29.0,0.0,0.0,25.0,0.0,0.0
2,Released,358.0,0.0,37.0,2.0,1.0,45.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()